In [1]:
import os
import pandas as pd
import datetime

def concatenate_xlsx_files(parent_folder):
    all_data = []

    for root, dirs, files in os.walk(parent_folder):
        if root != parent_folder:
            for file in files:
                if file.endswith('.xlsx'):
                    filepath = os.path.join(root, file)
                    try:
                        df = pd.read_excel(filepath, header=0)  # Read with the first row as header

                        # Skip the second row (empty row)
                        df = df.iloc[1:].copy()

                        df.reset_index(drop=True, inplace=True)

                        # Extract date from directory name (YYYYMMDD)
                        date_str = os.path.basename(root)

                        # Extract time from filename (hh-mm-ss)
                        time_str = file.split('_')[0]

                        # Combine date and time into the desired format
                        date_combined = f"{date_str}_{time_str}"
                        df['date'] = date_combined  # Add the 'date' column

                        all_data.append(df)

                    except pd.errors.EmptyDataError:
                        print(f"Skipping empty file: {filepath}")

    if not all_data:
        print("No suitable Excel files found in subdirectories or all files were empty after skipping rows.")
        return None

    # Ensure all expected columns are present before concatenation
    expected_columns = ["Linkedin Profile name", "Invitation acceptée", "Répondu", "Suite", "date"]
    for df in all_data:
        missing_columns = set(expected_columns) - set(df.columns)
        for col in missing_columns:
            df[col] = None  # Add missing columns with NaN values

    concatenated_df = pd.concat(all_data, ignore_index=True)

    # Set default values
    for col in ["Invitation acceptée", "Répondu", "Suite"]:
        concatenated_df[col] = concatenated_df[col].fillna("Pas encore")

    return concatenated_df

# Example usage:
parent_folder = "/home/arthurcornelio/code/arthurcornelio88/Linkedin-Auto-Connector/dfs"
concatenated_data = concatenate_xlsx_files(parent_folder)

if concatenated_data is not None:
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    output_filename = f'concatenated_output_{timestamp}.xlsx'  # Include timestamp in filename
    concatenated_data.to_excel(output_filename, index=False)
else:
    print("No data to concatenate. Skipping Excel file creation.")